In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("Basit_Regresyon") \
.config("spark.executor.memory","5g") \
.config("spark.driver.memory", "4g") \
.getOrCreate()

22/10/18 09:37:16 WARN Utils: Your hostname, batuhan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/18 09:37:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/18 09:37:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/18 09:37:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 1. Read The Dataset

In [3]:
df = spark.read.format("csv") \
.option("header", True) \
.option("sep",",") \
.option("inferSchema",True) \
.load("/home/batuhan/Desktop/datalar/Advertising2.csv")

In [4]:
df.toPandas().head(10)

,CO,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
5,6,8.7,48.9,75.0,7.2
6,7,57.5,32.8,23.5,11.8
7,8,120.2,19.6,11.6,13.2
8,9,8.6,2.1,1.0,4.8
9,10,199.8,2.6,21.2,10.6


In [5]:
#df = df.drop("CO")

In [6]:
df.toPandas().head(5)

,CO,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# VectorAssembler

In [7]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler() \
.setInputCols(["TV","Radio","Newspaper"]) \
.setOutputCol("features")

In [8]:
df2 = vector_assembler.transform(df)

In [9]:
df2.toPandas().head()

,CO,TV,Radio,Newspaper,Sales,features
0,1,230.1,37.8,69.2,22.1,"[230.1, 37.8, 69.2]"
1,2,44.5,39.3,45.1,10.4,"[44.5, 39.3, 45.1]"
2,3,17.2,45.9,69.3,9.3,"[17.2, 45.9, 69.3]"
3,4,151.5,41.3,58.5,18.5,"[151.5, 41.3, 58.5]"
4,5,180.8,10.8,58.4,12.9,"[180.8, 10.8, 58.4]"


In [10]:
import pandas as pd
import numpy as np

In [11]:
params = np.array((0.3,0,9))

# Regresion Model

In [12]:
from pyspark.ml.regression import LinearRegression
#linear_reg_obj = LinearRegression() \
#.setLabelCol("Sales") \
#.setFeaturesCol("features") \
#.setParams= params

In [13]:
linear_reg_obj = LinearRegression(featuresCol="features",labelCol="Sales",regParam=0.8)

# Train, Test Split

In [35]:
train_df, test_df = df2.randomSplit([0.8, 0.2], seed = 42)

# Training Model

In [36]:
model = linear_reg_obj.fit(train_df)

# Testing Model

In [37]:
result_df = model.transform(test_df)

In [38]:
result_df.toPandas().head(10)

,CO,TV,Radio,Newspaper,Sales,features,prediction
0,3,17.2,45.9,69.3,9.3,"[17.2, 45.9, 69.3]",12.557507
1,7,57.5,32.8,23.5,11.8,"[57.5, 32.8, 23.5]",11.818951
2,9,8.6,2.1,1.0,4.8,"[8.6, 2.1, 1.0]",4.780422
3,14,97.5,7.6,7.2,9.7,"[97.5, 7.6, 7.2]",9.319446
4,20,147.3,23.9,19.1,14.6,"[147.3, 23.9, 19.1]",14.027213
5,24,228.3,16.9,26.2,15.5,"[228.3, 16.9, 26.2]",16.252646
6,30,70.6,16.0,40.8,10.5,"[70.6, 16.0, 40.8]",9.776402
7,36,290.7,4.1,8.5,12.8,"[290.7, 4.1, 8.5]",16.638274
8,46,175.1,22.5,31.5,14.9,"[175.1, 22.5, 31.5]",15.013597
9,47,89.7,9.9,35.7,10.6,"[89.7, 9.9, 35.7]",9.547750


In [39]:
result_df.toPandas().head(20)

,CO,TV,Radio,Newspaper,Sales,features,prediction
0,3,17.2,45.9,69.3,9.3,"[17.2, 45.9, 69.3]",12.557507
1,7,57.5,32.8,23.5,11.8,"[57.5, 32.8, 23.5]",11.818951
2,9,8.6,2.1,1.0,4.8,"[8.6, 2.1, 1.0]",4.780422
3,14,97.5,7.6,7.2,9.7,"[97.5, 7.6, 7.2]",9.319446
4,20,147.3,23.9,19.1,14.6,"[147.3, 23.9, 19.1]",14.027213
5,24,228.3,16.9,26.2,15.5,"[228.3, 16.9, 26.2]",16.252646
6,30,70.6,16.0,40.8,10.5,"[70.6, 16.0, 40.8]",9.776402
7,36,290.7,4.1,8.5,12.8,"[290.7, 4.1, 8.5]",16.638274
8,46,175.1,22.5,31.5,14.9,"[175.1, 22.5, 31.5]",15.013597
9,47,89.7,9.9,35.7,10.6,"[89.7, 9.9, 35.7]",9.547750


In [40]:
model.coefficients

DenseVector([0.0407, 0.1598, 0.0063])

In [41]:
model.intercept

4.08827265871777

In [42]:
model.summary.r2

0.8839157065987285

In [43]:
model.summary.pValues

[0.0, 0.0, 0.3162260879131975, 0.0]

In [44]:
model.summary.tValues

[27.065681758114593, 17.514692802463635, 1.0052965085046834, 11.64329537706581]

In [45]:
model.summary.rootMeanSquaredError

1.7969624944528968

# y = 4.332336427374665 + 0.0391 * Advertisement
4.332336427374665 = model.intercept
0.0391 = model.coefficients
Lineer Regresyonu Şuan Formulize Ettik

# 100.000 TL'Lik Reklam Bütçesi Ayarlarsak Satışlarımıza Ne Kadar Etki Eder

In [25]:
df_predict_rdd = spark.sparkContext.parallelize([100.0])
df_predict = df_predict_rdd.map(lambda x: (x,)).toDF(["Advertisement2"])

In [26]:
df_predict.show()

+--------------+
|Advertisement2|
+--------------+
|         100.0|
+--------------+



In [27]:
vector_assembler2 = VectorAssembler() \
.setInputCols(["Advertisement2"]) \
.setOutputCol("features")

In [28]:
df_pred_vec = vector_assembler2.transform(df_predict)

In [29]:
df_pred_vec.show()

+--------------+--------+
|Advertisement2|features|
+--------------+--------+
|         100.0| [100.0]|
+--------------+--------+



In [30]:
df_pred_vec2 = df_pred_vec.drop("Advertisement2")

In [31]:
df_pred_vec2.show()

+--------+
|features|
+--------+
| [100.0]|
+--------+



In [32]:
#inear_reg_obj2 = LinearRegression(featuresCol="features",labelCol="new_predict",regParam=0.8)
#train_df, test_df = df_pred_vec2.randomSplit([0.8, 0.2], seed = 142)

In [33]:
#model2 = linear_reg_obj2.fit(train_df)

In [34]:
#model2.transform(df_pred_vec2).toPandas().head()